<a href="https://colab.research.google.com/github/Exabyte-io/api-examples/blob/dev/other/material/import_material_from_jarvis_db_entry.ipynb" target="_parent">
<img alt="Open in Google Colab" src="https://user-images.githubusercontent.com/20477508/128780728-491fea90-9b23-495f-a091-11681150db37.jpeg" width="150" border="0">
</a>

## Install Packages
First, install `express-py` which includes `jarvis-tools` as dependency.

In [4]:
!pip install express-py==2024.1.25.post7


## Get Materials Data From JARVIS
Then, let's get the dataset containing 2D materials from JARVIS and wrap it into a pandas dataframe.

In [5]:
import pandas as pd
from jarvis.db.figshare import data, get_jid_data

dft_2d=data('dft_2d')
df = pd.DataFrame(dft_2d)

Obtaining 2D dataset 1.1k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815705
Loading the zipfile...
Loading completed.


## Extract Structure and Convert to ESSE
Next, we extract an entry from the JARVIS dataset and convert it into ESSE format ready to be uploaded to Mat3ra.com.

In [6]:
import json
from express import ExPrESS

## Note: JVASP-670 is an entry for MoTe2
jarvis_db_entry = json.dumps(json.loads(df[(df['jid']=='JVASP-670')].to_json(orient='records'))[0])

kwargs = {
    "structure_string": jarvis_db_entry,
    "cell_type": "original",
    "structure_format": "jarvis-db-entry"
}

handler = ExPrESS("structure", **kwargs)
data = handler.property("material", **kwargs)

# To preview resulting JSON data
print(json.dumps(data, indent=4))

{
    "_id": "",
    "name": "material",
    "exabyteId": "",
    "hash": "",
    "formula": "Te2Mo",
    "unitCellFormula": "Te2 Mo1",
    "lattice": {
        "type": "HEX",
        "a": 3.562364004499092,
        "b": 3.562364004499092,
        "c": 35.346645,
        "alpha": 90.0,
        "beta": 90.0,
        "gamma": 120.00001186178349,
        "units": {
            "length": "angstrom",
            "angle": "degree"
        }
    },
    "basis": {
        "units": "crystal",
        "elements": [
            {
                "id": 1,
                "value": "Te"
            },
            {
                "id": 2,
                "value": "Te"
            },
            {
                "id": 3,
                "value": "Mo"
            }
        ],
        "coordinates": [
            {
                "id": 1,
                "value": [
                    0.6666669999999999,
                    0.3333330000000001,
                    0.05727934272677686
                

In [7]:
# @title Authorization Form
ACCOUNT_ID = "" # @param {type:"string"}
AUTH_TOKEN = ""  # @param {type:"string"}
MATERIALS_PROJECT_API_KEY = "MATERIALS_PROJECT_API_KEY"  # @param {type:"string"}
ORGANIZATION_ID = ""


import os

if "COLAB_JUPYTER_IP" in os.environ:
    os.environ.update(
        dict(
            ACCOUNT_ID=ACCOUNT_ID,
            AUTH_TOKEN=AUTH_TOKEN,
            MATERIALS_PROJECT_API_KEY=MATERIALS_PROJECT_API_KEY,
            ORGANIZATION_ID=ORGANIZATION_ID,
        )
    )

    !GIT_BRANCH="dev"; export GIT_BRANCH; curl -s "https://raw.githubusercontent.com/Exabyte-io/api-examples/${GIT_BRANCH}/scripts/env.sh" | bash


Traceback (most recent call last):
  File "/usr/local/bin/notebook-path", line 8, in <module>
    sys.exit(print_notebook_path())
  File "/usr/local/lib/python3.10/dist-packages/utils/notebook.py", line 46, in print_notebook_path
    print(get_notebook_info()["notebook_path"])
  File "/usr/local/lib/python3.10/dist-packages/utils/notebook.py", line 22, in get_notebook_info
    parsed = re.findall("(.*)/blob/(.*)/examples/(.*)", path)[0]
IndexError: list index out of range


In [10]:
from utils.settings import ENDPOINT_ARGS, ACCOUNT_ID
from utils.generic import wait_for_jobs_to_finish, get_property_by_subworkflow_and_unit_indicies, dataframe_to_html, display_JSON

import pandas as pd

# Relevant functions from the API client
from exabyte_api_client.endpoints.jobs import JobEndpoints
from exabyte_api_client.endpoints.projects import ProjectEndpoints
from exabyte_api_client.endpoints.materials import MaterialEndpoints
from exabyte_api_client.endpoints.bank_workflows import BankWorkflowEndpoints
from exabyte_api_client.endpoints.properties import PropertiesEndpoints
from exabyte_api_client.endpoints.workflows import WorkflowEndpoints

job_endpoints = JobEndpoints(*ENDPOINT_ARGS)
project_endpoints = ProjectEndpoints(*ENDPOINT_ARGS)
material_endpoints = MaterialEndpoints(*ENDPOINT_ARGS)
property_endpoints = PropertiesEndpoints(*ENDPOINT_ARGS)
bank_workflow_endpoints = BankWorkflowEndpoints(*ENDPOINT_ARGS)
workflow_endpoints = WorkflowEndpoints(*ENDPOINT_ARGS)

In [18]:
default_material = material_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]
default_workflow = workflow_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]

material_id = default_material["_id"]
workflow_id = default_workflow["_id"]
owner_id = default_material["owner"]["_id"]


material = material_endpoints.create(data)


In [22]:
material["lattice"]

{'a': 3.562364,
 'b': 3.562364,
 'c': 35.34665,
 'alpha': 90,
 'beta': 90,
 'gamma': 120,
 'units': {'length': 'angstrom', 'angle': 'degree'},
 'type': 'HEX',
 'vectors': {'a': [3.562364, 0, 0],
  'b': [-1.781182, 3.085098, 0],
  'c': [0, 0, 35.34665],
  'alat': 1,
  'units': 'angstrom'}}

In [23]:
config = {
    "owner": {"_id": owner_id},
    "_material": {"_id": material["_id"]},
    "workflow": {"_id": workflow_id},
    "name": "MoTe2 Job",
}


In [24]:
job = job_endpoints.create(config)
job_endpoints.submit(job["_id"])


In [ ]:
from utils.generic import wait_for_jobs_to_finish, get_property_by_subworkflow_and_unit_indicies
job_id = job["_id"]
wait_for_jobs_to_finish(job_endpoints, [job_id])


Wait for jobs to finish, poll interval: 10 sec
+---------------------+------------------+---------------+-----------------+----------------+
|        TIME         |   SUBMITTED-JOBS |   ACTIVE-JOBS |   FINISHED-JOBS |   ERRORED-JOBS |
+=====================+==================+===============+=================+================+
| 2024-02-02-17:29:10 |                1 |             0 |               0 |              0 |
+---------------------+------------------+---------------+-----------------+----------------+
+---------------------+------------------+---------------+-----------------+----------------+
|        TIME         |   SUBMITTED-JOBS |   ACTIVE-JOBS |   FINISHED-JOBS |   ERRORED-JOBS |
+=====================+==================+===============+=================+================+
| 2024-02-02-17:29:20 |                1 |             0 |               0 |              0 |
+---------------------+------------------+---------------+-----------------+----------------+
+------------

## Upload to Mat3ra.com
Finally, we can upload the material to Mat3ra.com using the REST API: follow the explanation in another example notebook [here](../../examples/material/create_material.ipynb). One can replace the content of the `CONFIG` variable with the JSON data above.